In [1]:
import pickle
import pandas as pd

with open('models/best_model_0.778.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

In [2]:
df_test = pd.DataFrame({
    'Hab': [10, 12, 14],
    'Bedrooms': [3, 1, 1],
    'Cleaning Fee':[40,20,30],
    'Bathrooms':[2,1,1],
    'Guests Included':[2,1,1],
    'Host Listings Count':[17,16,5]
})

In [3]:
predictions = model.predict(df_test)
print(predictions)

[106.62761273  61.90660482  60.11313789]


*Resumen: El modelo LightGBM fue el de mejor resultado r2 de 0.7784, aún asi el modelo no es capaz de generalizar y no se pudo replicar esos resultados,por lo que se debe continuar ajustando parámetros como min_data_in_leaf y bagging_fraction, evaluar la creación de nuevas features para mejorar aún más el rendimiento.*



In [6]:
from sklearn.metrics import r2_score, mean_squared_error
import preprocessing
df_train=pd.read_csv('./data/airbnb_train.csv', sep=";",decimal='.' )
df_test=pd.read_csv('./data/airbnb_test.csv', sep=";",decimal='.' )

df_train=df_train.dropna(subset=['Price'])
df_test=df_test.dropna(subset=['Price'])
df_train=preprocessing.filtrar_columnas(df_train,['Cleaning Fee','Accommodates','Room Type','Host Listings Count','Bathrooms','Bedrooms','Guests Included','Price','Beds'])
df_train , imputer , xnumeric = preprocessing.imputer_data_numeric_2(df_train)


df_train['Hab']=df_train['Beds']+df_train['Accommodates']
df_train=preprocessing.filtrar_columnas(df_train,['Cleaning Fee','Price','Host Listings Count','Guests Included','Bathrooms','Bedrooms','Hab'])
df_test[xnumeric] = imputer.transform(df_test[xnumeric])
df_test['Hab']=df_test['Beds']+df_test['Accommodates']
df_test = df_test[df_train.columns.intersection(df_test.columns)]


        
X_train = df_train.drop(columns=["Price"]) 
y_train = df_train["Price"] 
X_test = df_test.drop(columns=["Price"])
y_test = df_test["Price"]
        
y_pred = model.predict(X_test)


r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R² del modelo: {r2:.4f}")
print(f"Error cuadrático medio (MSE): {mse:.4f}")

R² del modelo: 0.7784
Error cuadrático medio (MSE): 1206.4821


**Error medio aprox 34.7 euros**